# 1. Install requirements

In [1]:
%pip install pandas
%pip install scikit-learn
%pip install xgboost
%pip install optuna
%pip install joblib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to us

# 2. Add column names to dataset

In [86]:
import pandas as pd

wilderness_columns = []
soil_columns = []
other_columns = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
]



for i in range(1,5):
    wilderness_columns += [f"Wilderness_Area_{i:02d}"]

for i in range(1,41):
    soil_columns += [f"Soil_Type_{i:02d}"]

column_names = other_columns + wilderness_columns + soil_columns + ["Cover_Type"]

df = pd.read_csv("covtype.csv", header=None)

df.columns = column_names

df['Cover_Type'] = df['Cover_Type'] - 1


## Ensure wilderness areas and soil types columns are of type one-hot

In [88]:

# Wilderness areas columns can have only 4 possible combinations
wilderness = df[wilderness_columns].drop_duplicates()
if len(wilderness) == 4:
    print("Wilderness areas columns are ok.")
else:
    print("Wilderness areas columns are NOT ok.")

# Soil types columns can have only 40 possible combinations
soil = df[soil_columns].drop_duplicates()
if len(soil) == 40:
    print("Soil types columns are ok.")
else:
    print("Soil type columns are NOT ok.")

Wilderness areas columns are ok.
Soil types columns are ok.


## Show some stats on the other columns to check is there's something missing

In [357]:
display(df[other_columns].describe())
display(df['Cover_Type'].value_counts())

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000


Cover_Type
1    283301
0    211840
2     35754
6     20510
5     17367
4      9493
3      2747
Name: count, dtype: int64

Everything looks fine. EDA finished

## Just for fun: create vectors of a given size

In [140]:
import numpy as np

rng = np.random.default_rng()

def generate_random_one_hot_sets(n_sets=1, size=4):
    random_vectors = np.zeros((n_sets, size), dtype=int)

    random_indices = rng.integers(low=0, high=size, size=n_sets)

    row_indices = np.arange(n_sets)
    random_vectors[row_indices, random_indices] = 1

    return random_vectors

def generate_random_number(min=0, max=10000):
    return rng.integers(low=min, high=max)

# 3. Train XGBoost

In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['Cover_Type'])
y = df['Cover_Type']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
scaler = StandardScaler()

xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    n_estimators=100,             # Tree's number (estimators)
    learning_rate=0.1,            # Learning rate
    max_depth=6,                  # Max. depth
    eval_metric='merror',         # Classification error
    random_state=42,
    tree_method='hist',          
    device="cuda"                 # Use this if you have NVidia
)

print("Trining XGBoost model...")
xgb_model.fit(scaler.fit_transform(X_train), y_train)
print("Training finished.")

y_pred = xgb_model.predict(scaler.transform(X_test))

Trining XGBoost model...
Training finished.


/home/vscode/.local/lib/python3.12/site-packages/xgboost/core.py:729: UserWarning: [18:42:11] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


# 4. Show XGBoost metrics

In [3]:
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy of XGBoost model: {accuracy:.4f}")

print("\nClassification report:")
print(classification_report(y_test, y_pred))


Accuracy of XGBoost model: 0.8108

Classification report:
              precision    recall  f1-score   support

           0       0.81      0.78      0.79     42368
           1       0.81      0.86      0.84     56661
           2       0.79      0.86      0.82      7151
           3       0.83      0.83      0.83       549
           4       0.87      0.31      0.46      1899
           5       0.73      0.49      0.59      3473
           6       0.90      0.81      0.85      4102

    accuracy                           0.81    116203
   macro avg       0.82      0.71      0.74    116203
weighted avg       0.81      0.81      0.81    116203



# 5. Optimize XGBoost

In [7]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from optuna.pruners import MedianPruner
from optuna import TrialPruned
from xgboost.callback import EarlyStopping


def objective(trial, X, y):
    # Hyperparameters
    param = {
        'objective': 'multi:softmax',
        'eval_metric': 'merror',
        'n_jobs': -1,
        'random_state': 42,
        'tree_method': 'hist',
        'device': 'cuda',

        # Learning rate
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),

        # Overfitting control
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),

        # Bagging-like
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),

        # L1 and L2 regularization
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),  # L2
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True)   # L1
    }

    # Estimators (trees)
    n_estimators = 500

    # Create kfold
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    scaler = StandardScaler()

    # callback_es = EarlyStopping(
    #     rounds=20,
    #     metric_name="merror",
    #     data_name="validation_0"
    # )

    fold_step = 0
    for train_index, val_index in kf.split(X, y):
        X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
        y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled = scaler.transform(X_val_fold)

        # Create model
        model = xgb.XGBClassifier(
            **param,
            n_estimators=n_estimators
        )

        # Train model
        model.fit(
            X_train_scaled, y_train_fold,
            eval_set=[(X_val_scaled, y_val_fold)],
            verbose=False # No progress
        )

        # Predict and compute precission
        y_pred = model.predict(X_val_scaled)
        accuracy = accuracy_score(y_val_fold, y_pred)
        accuracies.append(accuracy)

        fold_step += 1
        trial.report(np.mean(accuracies), fold_step)
        if trial.should_prune():
            raise TrialPruned

    # Return the mean of all accuracies to optuna (maximize)
    return np.mean(accuracies)


# Create optuna study
study = optuna.create_study(
    direction='maximize',
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=30, interval_steps=10)
)

# Run optimization (it can take a long time)
print("Optuna study is starting...")
study.optimize(
    lambda trial: objective(trial, X_train, y_train),
    n_trials=50, 
    show_progress_bar=True
)

print("\n--- Optuna Results---")
print(f"Best score for cross validation: {study.best_value:.4f}")
print("Best hyperparams:")
print(study.best_params)

/home/vscode/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-15 16:49:35,758] A new study created in memory with name: no-name-cfafa8fd-19d6-4e53-a9b7-3d0f110a11c9


Optuna study is starting...


Best trial: 0. Best value: 0.958678:   2%|▏         | 1/50 [01:57<1:35:44, 117.24s/it]

[I 2025-10-15 16:51:32,997] Trial 0 finished with value: 0.9586776501667901 and parameters: {'learning_rate': 0.2863414865272773, 'max_depth': 8, 'min_child_weight': 9, 'gamma': 8.325937909892682e-06, 'subsample': 0.9048967594249374, 'colsample_bytree': 0.7115815178224135, 'lambda': 3.170849864792997e-07, 'alpha': 0.04137263265173337}. Best is trial 0 with value: 0.9586776501667901.


Best trial: 0. Best value: 0.958678:   4%|▍         | 2/50 [03:03<1:09:34, 86.96s/it] 

...

[I 2025-10-15 18:27:41,866] Trial 47 finished with value: 0.9403217235177843 and parameters: {'learning_rate': 0.054847436631770226, 'max_depth': 10, 'min_child_weight': 3, 'gamma': 6.450754945106557e-08, 'subsample': 0.8485894825746234, 'colsample_bytree': 0.5309913059960999, 'lambda': 3.559690491379976e-07, 'alpha': 4.277969581026634e-05}. Best is trial 46 with value: 0.9666336062323072.


Best trial: 46. Best value: 0.966634:  98%|█████████▊| 49/50 [1:39:08<01:56, 116.20s/it]

[I 2025-10-15 18:28:43,876] Trial 48 finished with value: 0.8435400375630309 and parameters: {'learning_rate': 0.12867512769976844, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.0020667796463782165, 'subsample': 0.980103040269324, 'colsample_bytree': 0.8724985545336613, 'lambda': 5.185592016847184e-06, 'alpha': 0.00010274863216333713}. Best is trial 46 with value: 0.9666336062323072.


Best trial: 46. Best value: 0.966634: 100%|██████████| 50/50 [1:41:44<00:00, 122.09s/it]

[I 2025-10-15 18:31:20,262] Trial 49 finished with value: 0.9659860284146357 and parameters: {'learning_rate': 0.1763558289078166, 'max_depth': 10, 'min_child_weight': 4, 'gamma': 7.427196859737999e-07, 'subsample': 0.8679044623364404, 'colsample_bytree': 0.8189472652871326, 'lambda': 3.1790377638050322e-06, 'alpha': 0.0010521032775424257}. Best is trial 46 with value: 0.9666336062323072.

--- Optuna Results---
Best score for cross validation: 0.9666
Best hyperparams:
{'learning_rate': 0.1786912232090324, 'max_depth': 10, 'min_child_weight': 3, 'gamma': 6.009415529412678e-08, 'subsample': 0.8558726329105429, 'colsample_bytree': 0.8641884002688407, 'lambda': 2.3812223464914405e-08, 'alpha': 4.630223688020337e-05}


# 6. Train XGBoost model with optimized parameters

In [4]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

fixed_params = {
    'objective': 'multi:softmax',
    'eval_metric': 'merror',
    'n_jobs': -1,
    'random_state': 42,
    'tree_method': 'hist',
    'device': 'cuda'
}

# Use this if previous step fails
study = {'learning_rate': 0.1786912232090324, 'max_depth': 10, 'min_child_weight': 3, 'gamma': 6.009415529412678e-08, 'subsample': 0.8558726329105429, 'colsample_bytree': 0.8641884002688407, 'lambda': 2.3812223464914405e-08, 'alpha': 4.630223688020337e-05}
final_params = { **fixed_params, **study }

# Use this if previous step don't fail
# final_params = { **fixed_params, **study.best_params }

final_n_estimators = 1000
scaler = StandardScaler()

final_model = xgb.XGBClassifier(
    **final_params,
    n_estimators = final_n_estimators
)

print("\nTraining optimized model with train data...")
final_model.fit(scaler.fit_transform(X_train), y_train)

y_pred = final_model.predict(scaler.transform(X_test))


Training optimized model with train data...


# 7. Show performance of optimized XGBoost

In [5]:
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy for optimized XGBoost model: {accuracy:.4f}")

print("\nClassification report:")
print(classification_report(y_test, y_pred))


Accuracy for optimized XGBoost model: 0.9707

Classification report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     42368
           1       0.97      0.98      0.98     56661
           2       0.96      0.97      0.97      7151
           3       0.90      0.87      0.89       549
           4       0.93      0.90      0.91      1899
           5       0.95      0.93      0.94      3473
           6       0.97      0.97      0.97      4102

    accuracy                           0.97    116203
   macro avg       0.95      0.94      0.95    116203
weighted avg       0.97      0.97      0.97    116203



# 8. Train the model with all data and save it

In [6]:
import joblib
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

print("\nTraining optimized model with all data...")
final_model.fit(scaler.fit_transform(X), y)

print("\nSaving model and scaler...")
joblib.dump(final_model, 'xgboost_optimized.pkl')
joblib.dump(scaler, 'xgboost_scaler.pkl')


Training optimized model with all data...

Saving model and scaler...


['xgboost_scaler.pkl']

# 9. Play with the model

In [348]:
import joblib

model = joblib.load('xgboost_optimized.pkl')
scaler = joblib.load('xgboost_scaler.pkl')

elevation = generate_random_number(1000, 1500)
aspect = generate_random_number(0, 360)
slope = generate_random_number(0, 75)
hdh = generate_random_number(0, 10000)
vdh = generate_random_number(0, 5000)
hdr = generate_random_number(0, 10000)
h9 = generate_random_number(0,254)
h12 = generate_random_number(0,254)
h15 = generate_random_number(0,254)
hdfp = generate_random_number(0,10000)
wilderness = generate_random_one_hot_sets(1, 4)
soil = generate_random_one_hot_sets(1, 40)

df_other = pd.DataFrame(columns=other_columns)
df_other.loc[1]=[elevation, aspect, slope, hdh, vdh, hdr, h9, h12, h15, hdfp]

df_wilderness = pd.DataFrame(columns=wilderness_columns)
df_wilderness.loc[1] = wilderness[0]

df_soil = pd.DataFrame(columns=soil_columns)
df_soil.loc[1] = soil[0]

X_rnd = pd.concat([df_other, df_wilderness, df_soil], axis=1)
y_pred = model.predict(scaler.transform(X_rnd))

display(X_rnd)
print(f"Cover predicted: {y_pred[0] + 1}")


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40
1,1238,18,26,482,878,9534,112,213,171,9736,...,0,0,0,0,0,0,0,0,0,0


Cover predicted: 3
